# coc case study

This case study concerns the transient difference absorption measured in the visible spectral region after 530 nm excitation of an articial light harvester, named calix-orange-calix (coc), in the solvent toluene. The measurement, the global analysis and the target analysis is based upon the model explained in ([Hippius et al 2007][Hippius2007]).

[Hippius2007]: https://www.few.vu.nl/~ivo/pub/2007/HippiusWilliamsWuerthnerJPCC111_13988_13996.pdf "C. Hippius, I.H.M. van Stokkum, E. Zangrando, R.M. Williams, F. Würthner, Excited state interactions in calix[4]arene-perylene bisimide dye conjugates: Global and target analysis of supramolecular building blocks, Journal of Physical Chemistry C, 111 (2007) 13988-13996."

### Inspect data

In [ ]:
from pyglotaran_extras import plot_data_overview

coc_tol_data_path = "2016coc_tol.ascii"
plot_data_overview(coc_tol_data_path, nr_of_data_svd_vectors=5, linlog=True, linthresh=1);

## Global Analysis: sequential scheme with decay rates in decreasing order

### Used model and parameters

In [ ]:
from glotaran.utils.ipython import display_file

coc_tol_global_model_path = "models/coc_tol-global-model.yml"
coc_tol_global_parameters_path = "models/coc_tol-global-parameters.yml"

#### Model file

In [ ]:
display_file(coc_tol_global_model_path, syntax="yaml")

#### Parameters file

The free parameters are the center (the location of the maximum of the IRF at 550 nm, the dispersion_center) and the width of the IRF, and the kinetic parameters. The final state is long lived. The final decay rate has been fixed at a very small number to avoid numerical problems with the optimization.
The dispersion parameters have been fixed as well, but could have been estimated from the data.

Exercise 5: estimate the missing starting values for the parameters from the data inspection, and insert these in the models/coc_tol-global-parameters.yml file.

In [ ]:
display_file(coc_tol_global_parameters_path, syntax="yaml")

### Create scheme and optimize it

In [ ]:
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme

coc_tol_global_scheme = Scheme(
    model=coc_tol_global_model_path,
    parameters=coc_tol_global_parameters_path,
    maximum_number_function_evaluations=9,
    data={"coc_tol_data": coc_tol_data_path},
)
coc_tol_global_scheme.validate()

In [ ]:
coc_tol_global_result = optimize(coc_tol_global_scheme)

In [ ]:
coc_tol_global_result

In [ ]:
coc_tol_global_result.optimized_parameters

In [ ]:
from show_a_matrix import show_a_matrix

show_a_matrix(coc_tol_global_result)

### Result plots

#### Fit quality

In [ ]:
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths

wavelengths = select_plot_wavelengths(coc_tol_global_result.data["coc_tol_data"])
plot_fitted_traces(
    coc_tol_global_result.data["coc_tol_data"], wavelengths, linlog=True, linthresh=1
);

Note that in this plot the time axis is linear until 1 ps and logarithmic thereafter. This plot is more informative.

#### Overview

In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(
    coc_tol_global_result.data["coc_tol_data"],
    linlog=True,
    figure_only=False,
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
);

Exercise 6: Comment on the interpretation of the EADS, with the help of literature. Which EADS is the most difficult to interpret?

## First Target Analysis: small triplet yield

### Used model and parameters

In [ ]:
coc_tol_target_model_path = "models/coc_tol-target-model.yml"
coc_tol_target_parameters_path = "models/coc_tol-target-parameters.yml"

#### Model file

In [ ]:
display_file(coc_tol_target_model_path, syntax="yaml")

#### Parameters file

Exercise 7: In the regular sequential scheme (with decay rates in decreasing order) the final EADS was very small. This can be interpreted as the generation of a triplet with a small yield.
Insert the missing starting values in the models/coc_tol-target-parameters.yml file. Hint: use the above estimated kinetic and IRF parameters. Study the parameters file carefully. Try to find a better Tyield value, such that the bleaches near 530 nm of the 3rd and 4th SADS are comparable.

In [ ]:
display_file(coc_tol_target_parameters_path, syntax="yaml")

### Create scheme and optimize it

In [ ]:
coc_tol_target_scheme = Scheme(
    model=coc_tol_target_model_path,
    parameters=coc_tol_target_parameters_path,
    maximum_number_function_evaluations=3,
    data={"coc_tol_data": coc_tol_data_path},
)
coc_tol_target_scheme.validate()

In [ ]:
coc_tol_target_result = optimize(coc_tol_target_scheme)

In [ ]:
coc_tol_target_result

### Result plots

#### Fit quality

In [ ]:
fig_target, axes = plot_fitted_traces(
    coc_tol_target_result.data["coc_tol_data"], wavelengths, linlog=True, linthresh=1
);

#### Overview

In [ ]:
plot_overview(
    coc_tol_target_result.data["coc_tol_data"],
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
    linlog=True,
    linthresh=1,
);

Exercise 8: Comment on the interpretation of the SADS, with the help of literature. Which SADS is the most difficult to interpret?

In [ ]:
coc_tol_target_result.optimized_parameters

In [ ]:
compartments = coc_tol_target_scheme.model.initial_concentration["input2"].compartments

coc_tol_target_scheme.model.k_matrix["km2"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
)

In [ ]:
show_a_matrix(coc_tol_target_result)